In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

movies = pd.read_csv("/Users/komalkumari/Downloads/ml-latest-small/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
movies["genres_clean"] = movies["genres"].str.replace("|", " ")
movies[["title", "genres_clean"]].head()
movies = movies[movies["genres"] != "(no genres listed)"]

/var/folders/td/65qq71914851dbgx1d5y11mh0000gn/T/ipykernel_1525/632978288.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movies["genres_clean"] = movies["genres"].str.replace("|", " ")


In [4]:
vectorizer = CountVectorizer(stop_words = "english")
genre_matrix = vectorizer.fit_transform(movies["genres_clean"])

genre_matrix.shape

(9708, 21)

In [5]:
cosine_sim = cosine_similarity(genre_matrix, genre_matrix)
cosine_sim

array([[1.        , 0.77459667, 0.31622777, ..., 0.        , 0.31622777,
        0.4472136 ],
       [0.77459667, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.31622777, 0.        , 1.        , ..., 0.        , 0.        ,
        0.70710678],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.31622777, 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.4472136 , 0.        , 0.70710678, ..., 0.        , 0.        ,
        1.        ]])

In [6]:
movies_indices = pd.Series(movies.index, index = movies["title"])

In [7]:
def recommended_movies(title, cosine_sim = cosine_sim , n = 10):
    idx = movies_indices[title]
    similarity_scores = list(enumerate(cosine_sim[idx]))
    similarity_scores = sorted(similarity_scores, key = lambda x : x [1], reverse = True)
    top_movies = similarity_scores[1:n+1]
    movies_indices_list = [i[0] for i in top_movies]
    return movies["title"].iloc[movies_indices_list]

In [8]:
recommended_movies("Jumanji (1995)")

53                     Indian in the Cupboard, The (1995)
109                     NeverEnding Story III, The (1994)
767                       Escape to Witch Mountain (1975)
1514            Darby O'Gill and the Little People (1959)
1556                                  Return to Oz (1985)
1617                        NeverEnding Story, The (1984)
1618    NeverEnding Story II: The Next Chapter, The (1...
1799                        Santa Claus: The Movie (1985)
3574    Harry Potter and the Sorcerer's Stone (a.k.a. ...
6075    Chronicles of Narnia: The Lion, the Witch and ...
Name: title, dtype: object